Vado a definire i diversi import che ci saranno all'interno del progetto

In [1]:
#import per la gestione dei
import cv2
import tensorflow as tf
import numpy as np
import os 
from matplotlib import pyplot as plt
import time 

#import per la divisione del dataset
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

import ffmpeg


In [2]:
cap = cv2.VideoCapture(2) #passiamo il percorso di un mp4 al posto dello 0 per lavorare sul video
i=0
# Load the input image.

while cap.isOpened():
    ret,frame = cap.read()
    
    cv2.imshow('MoveNet', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()


In [2]:
from IPython.display import HTML, display

In [3]:
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}
cyan = (255, 255, 0)
magenta = (255, 0, 255)

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): magenta,
    (0, 2): cyan,
    (1, 3): magenta,
    (2, 4): cyan,
    (0, 5): magenta,
    (0, 6): cyan,
    (5, 7): magenta,
    (7, 9): cyan,
    (6, 8): magenta,
    (8, 10): cyan,
    (5, 6): magenta,
    (5, 11): cyan,
    (6, 12): magenta,
    (11, 12): cyan,
    (11, 13): magenta,
    (13, 15): cyan,
    (12, 14): magenta,
    (14, 16): cyan
}


def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))
  return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot


#def to_gif(images, fps):
 # """Converts image sequence (4D numpy array) to gif."""
  #imageio.mimsave('./animation.gif', images, fps=fps)
  #return embed.embed_file('./animation.gif')

In [4]:
interpreter = tf.lite.Interpreter(model_path="lite-model_movenet_singlepose_lightning_3 (1).tflite")
interpreter.allocate_tensors()

def movenet(input_image):
  """Runs detection on an input image.

  Args:
    input_image: A [1, height, width, 3] tensor represents the input image
      pixels. Note that the height/width should already be resized and match the
      expected input resolution of the model before passing into this function.

  Returns:
    A [1, 1, 17, 3] float numpy array representing the predicted keypoint
    coordinates and scores.
  """
  # TF Lite format expects tensor type of uint8.
  input_image = tf.cast(input_image, dtype = tf.float32)
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
  # Invoke inference.
  interpreter.invoke()
  # Get the model prediction.
  keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
  return keypoints_with_scores

In [5]:
# Confidence score to determine whether a keypoint prediction is reliable.
MIN_CROP_KEYPOINT_SCORE = 0.2

def init_crop_region(image_height, image_width):
  """Defines the default crop region.

  The function provides the initial crop region (pads the full image from both
  sides to make it a square image) when the algorithm cannot reliably determine
  the crop region from the previous frame.
  """
  if image_width > image_height:
    box_height = image_width / image_height
    box_width = 1.0
    y_min = (image_height / 2 - image_width / 2) / image_height
    x_min = 0.0
  else:
    box_height = 1.0
    box_width = image_height / image_width
    y_min = 0.0
    x_min = (image_width / 2 - image_height / 2) / image_width

  return {
    'y_min': y_min,
    'x_min': x_min,
    'y_max': y_min + box_height,
    'x_max': x_min + box_width,
    'height': box_height,
    'width': box_width
  }

def torso_visible(keypoints):
  """Checks whether there are enough torso keypoints.

  This function checks whether the model is confident at predicting one of the
  shoulders/hips which is required to determine a good crop region.
  """
  return ((keypoints[0, 0, KEYPOINT_DICT['left_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE) and
          (keypoints[0, 0, KEYPOINT_DICT['left_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE))

def determine_torso_and_body_range(
    keypoints, target_keypoints, center_y, center_x):
  """Calculates the maximum distance from each keypoints to the center location.

  The function returns the maximum distances from the two sets of keypoints:
  full 17 keypoints and 4 torso keypoints. The returned information will be
  used to determine the crop size. See determineCropRegion for more detail.
  """
  torso_joints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
  max_torso_yrange = 0.0
  max_torso_xrange = 0.0
  for joint in torso_joints:
    dist_y = abs(center_y - target_keypoints[joint][0])
    dist_x = abs(center_x - target_keypoints[joint][1])
    if dist_y > max_torso_yrange:
      max_torso_yrange = dist_y
    if dist_x > max_torso_xrange:
      max_torso_xrange = dist_x

  max_body_yrange = 0.0
  max_body_xrange = 0.0
  for joint in KEYPOINT_DICT.keys():
    if keypoints[0, 0, KEYPOINT_DICT[joint], 2] < MIN_CROP_KEYPOINT_SCORE:
      continue
    dist_y = abs(center_y - target_keypoints[joint][0]);
    dist_x = abs(center_x - target_keypoints[joint][1]);
    if dist_y > max_body_yrange:
      max_body_yrange = dist_y

    if dist_x > max_body_xrange:
      max_body_xrange = dist_x

  return [max_torso_yrange, max_torso_xrange, max_body_yrange, max_body_xrange]

def determine_crop_region(
      keypoints, image_height,
      image_width):
  """Determines the region to crop the image for the model to run inference on.

  The algorithm uses the detected joints from the previous frame to estimate
  the square region that encloses the full body of the target person and
  centers at the midpoint of two hip joints. The crop size is determined by
  the distances between each joints and the center point.
  When the model is not confident with the four torso joint predictions, the
  function returns a default crop which is the full image padded to square.
  """
  target_keypoints = {}
  for joint in KEYPOINT_DICT.keys():
    target_keypoints[joint] = [
      keypoints[0, 0, KEYPOINT_DICT[joint], 0] * image_height,
      keypoints[0, 0, KEYPOINT_DICT[joint], 1] * image_width
    ]

  if torso_visible(keypoints):
    center_y = (target_keypoints['left_hip'][0] +
                target_keypoints['right_hip'][0]) / 2;
    center_x = (target_keypoints['left_hip'][1] +
                target_keypoints['right_hip'][1]) / 2;

    (max_torso_yrange, max_torso_xrange,
      max_body_yrange, max_body_xrange) = determine_torso_and_body_range(
          keypoints, target_keypoints, center_y, center_x)

    crop_length_half = np.amax(
        [max_torso_xrange * 1.9, max_torso_yrange * 1.9,
          max_body_yrange * 1.2, max_body_xrange * 1.2])

    tmp = np.array(
        [center_x, image_width - center_x, center_y, image_height - center_y])
    crop_length_half = np.amin(
        [crop_length_half, np.amax(tmp)]);

    crop_corner = [center_y - crop_length_half, center_x - crop_length_half];

    if crop_length_half > max(image_width, image_height) / 2:
      return init_crop_region(image_height, image_width)
    else:
      crop_length = crop_length_half * 2;
      return {
        'y_min': crop_corner[0] / image_height,
        'x_min': crop_corner[1] / image_width,
        'y_max': (crop_corner[0] + crop_length) / image_height,
        'x_max': (crop_corner[1] + crop_length) / image_width,
        'height': (crop_corner[0] + crop_length) / image_height -
            crop_corner[0] / image_height,
        'width': (crop_corner[1] + crop_length) / image_width -
            crop_corner[1] / image_width
      }
  else:
    return init_crop_region(image_height, image_width)

def crop_and_resize(image, crop_region, crop_size):
  """Crops and resize the image to prepare for the model input."""
  boxes=[[crop_region['y_min'], crop_region['x_min'],
          crop_region['y_max'], crop_region['x_max']]]
  output_image = tf.image.crop_and_resize(
      image, box_indices=[0], boxes=boxes, crop_size=crop_size)
  return output_image

def run_inference(movenet, image, crop_region, crop_size):
  """Runs model inferece on the cropped region.

  The function runs the model inference on the cropped region and updates the
  model output to the original image coordinate system.
  """
  image_height, image_width, _ = image.shape
  input_image = crop_and_resize(
    tf.expand_dims(image, axis=0), crop_region, crop_size=crop_size)
  # Run model inference.
  keypoints_with_scores = movenet(input_image)
  # Update the coordinates.
  for idx in range(17):
    keypoints_with_scores[0, 0, idx, 0] = (
        crop_region['y_min'] * image_height +
        crop_region['height'] * image_height *
        keypoints_with_scores[0, 0, idx, 0]) / image_height
    keypoints_with_scores[0, 0, idx, 1] = (
        crop_region['x_min'] * image_width +
        crop_region['width'] * image_width *
        keypoints_with_scores[0, 0, idx, 1]) / image_width
  return keypoints_with_scores

In [6]:
image_path= "/progetto_tesi/dataset/Cobra/Abhay_Bhuj.mp4"
image_height= 768
image_width = 1366

In [7]:
try:
    out, _ = (

        ffmpeg
        .input(image_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24')
        .run(capture_stdout=True, capture_stderr=True)

    )
except ffmpeg.Error as e:
    print('stdout:', e.stdout.decode('utf8'))
    print('stderr:', e.stderr.decode('utf8'))
    raise e
image = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, image_height, image_width, 3])
)

In [8]:
input_size=192

In [9]:
image[2, :, :, :]

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 1, 6],
        [0, 1, 6],
        [0, 1, 6],
        ...,
        [0, 3, 7],
        [0, 3, 7],
        [0, 3, 7]],

       [[0, 1, 6],
        [0, 1, 6],
        [0, 1, 6],
        ...,
        [0, 3, 7],
        [0, 3, 7],
        [0, 3, 7]],

       [[0, 1, 6],
        [0, 1, 6],
        [0, 1, 6],
        ...,
        [0, 3, 7],
        [0, 3, 7],
        [0, 3, 7]]], dtype=uint8)

In [10]:
# Load the input image.
num_frames, image_height, image_width, _ = image.shape
crop_region = init_crop_region(image_height, image_width)

output_images = []
for frame_idx in range(num_frames):
  keypoints_with_scores = run_inference(
      movenet, image[frame_idx, :, :, :], crop_region,
      crop_size=[input_size, input_size])
  
  crop_region = determine_crop_region(
      keypoints_with_scores, image_height, image_width)
  

# Prepare gif visualization.
print(keypoints_with_scores)

[[[[0.753092   0.63755035 0.54855585]
   [0.74005777 0.6396894  0.4352818 ]
   [0.7394423  0.6381791  0.59331244]
   [0.72524244 0.62574285 0.4093228 ]
   [0.72440463 0.6230626  0.602445  ]
   [0.7182283  0.5981377  0.55981666]
   [0.7315869  0.5978891  0.70037377]
   [0.70428073 0.5476814  0.382074  ]
   [0.7393409  0.54551387 0.3259777 ]
   [0.7613647  0.55591834 0.2687764 ]
   [0.8096967  0.5511092  0.2906036 ]
   [0.7373184  0.49829748 0.7415344 ]
   [0.755394   0.49590078 0.7363815 ]
   [0.74610025 0.44113764 0.31730747]
   [0.7743585  0.42332745 0.5527617 ]
   [0.71098065 0.37705398 0.3736285 ]
   [0.7624408  0.35175636 0.55789816]]]]


In [11]:
print(num_frames)

545


In [12]:
print(keypoints_with_scores)

[[[[0.753092   0.63755035 0.54855585]
   [0.74005777 0.6396894  0.4352818 ]
   [0.7394423  0.6381791  0.59331244]
   [0.72524244 0.62574285 0.4093228 ]
   [0.72440463 0.6230626  0.602445  ]
   [0.7182283  0.5981377  0.55981666]
   [0.7315869  0.5978891  0.70037377]
   [0.70428073 0.5476814  0.382074  ]
   [0.7393409  0.54551387 0.3259777 ]
   [0.7613647  0.55591834 0.2687764 ]
   [0.8096967  0.5511092  0.2906036 ]
   [0.7373184  0.49829748 0.7415344 ]
   [0.755394   0.49590078 0.7363815 ]
   [0.74610025 0.44113764 0.31730747]
   [0.7743585  0.42332745 0.5527617 ]
   [0.71098065 0.37705398 0.3736285 ]
   [0.7624408  0.35175636 0.55789816]]]]


In [13]:
def draw_keypoints(frame, keypoints, threshold):
        
    y, x, c = frame.shape
    # Denormalize the coordinates : multiply the normalized coordinates by the input_size(width,height)
    denormalized_coordinates = np.squeeze(np.multiply(keypoints, [y,x,1]))
    # Iterate through the points
    for keypoint in denormalized_coordinates:
        # Unpack the keypoint values : y, x, confidence score
        keypoint_y, keypoint_x, keypoint_confidence = keypoint
        if keypoint_confidence > threshold:
     
            cv2.circle(
                img=frame, 
                center=(int(keypoint_x), int(keypoint_y)), 
                radius=4, 
                color=(255,0,0),
                thickness=-1
            )

In [18]:
def draw_edges(denormalized_coordinates,keypoints, frame, edges_colors, threshold):
    
    y,x,c = frame.shape
    denormalized_coordinates = np.squeeze(np.multiply(keypoints, [y,x,1]))
    # Iterate through the edges 
    for edge, color in edges_colors.items():
        # Get the dict value associated to the actual edge
        p1, p2 = edge
        # Get the points
        y1, x1, confidence_1 = denormalized_coordinates[p1]
        y2, x2, confidence_2 = denormalized_coordinates[p2]
        # Draw the line from point 1 to point 2, the confidence > threshold
        if (confidence_1 > threshold) & (confidence_2 > threshold):      
            cv2.line(
                img=frame, 
                pt1=(int(x1), int(y1)),
                pt2=(int(x2), int(y2)), 
                color=(255,255,255), 
                thickness=2, 
                lineType=cv2.LINE_AA # Gives anti-aliased (smoothed) line which looks great for curves
            )

In [15]:
def draw_skeleton(frame, keypoints_with_scores, threshold):
        draw_keypoints(frame , keypoints_with_scores, threshold)
        draw_edges(frame,keypoints_with_scores, frame, KEYPOINT_EDGE_INDS_TO_COLOR, threshold)

In [16]:
cap = cv2.VideoCapture('/progetto_tesi/dataset/Cobra/Abhay_Bhuj.mp4') #passiamo il percorso di un mp4 al posto dello 0 per lavorare sul video
i=0
# Load the input image.
num_frames, image_height, image_width, _ = image.shape
crop_region = init_crop_region(image_height, image_width)
for frame_idx in range(num_frames):
    ret,frame = cap.read()


    keypoints_with_scores = run_inference(
    movenet, image[frame_idx, :, :, :], crop_region,
    crop_size=[input_size, input_size])

    crop_region = determine_crop_region(
    keypoints_with_scores, image_height, image_width)
    i+=1
    print(i)
    print(keypoints_with_scores)
    draw_skeleton(frame, keypoints_with_scores, 0.3)
    cv2.imshow('MoveNet', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
print(keypoints_with_scores)

1
[[[[0.78848475 0.63525486 0.15182328]
   [0.78929013 0.6334259  0.1549647 ]
   [0.7886955  0.6337419  0.16301948]
   [0.78737104 0.62616    0.13715726]
   [0.78771424 0.62309116 0.1292514 ]
   [0.7928403  0.6254313  0.13182934]
   [0.76241183 0.4820612  0.06131705]
   [0.78221965 0.6464685  0.12227806]
   [0.7667824  0.49172008 0.05018386]
   [0.8120146  0.6408323  0.12072406]
   [0.7980571  0.59907794 0.06771949]
   [0.81418115 0.59028935 0.05330569]
   [0.8094963  0.53068906 0.04025159]
   [0.7930045  0.6510184  0.08000186]
   [0.8639054  0.68370366 0.09073988]
   [0.8357761  0.65090215 0.07270958]
   [0.8784377  0.6738176  0.066517  ]]]]
2
[[[[0.77452344 0.49885875 0.06251176]
   [0.78753287 0.6300658  0.14987235]
   [0.7776495  0.49416798 0.06475613]
   [0.78551316 0.62478673 0.13372593]
   [0.775055   0.4854902  0.04845059]
   [0.74224967 0.50321007 0.07124259]
   [0.73840916 0.46948117 0.05190194]
   [0.7785847  0.6439942  0.12344521]
   [0.76511997 0.4878884  0.06743604]
   [0

In [20]:
frame.shape

(480, 640, 3)

In [22]:
cap = cv2.VideoCapture(0) #passiamo il percorso di un mp4 al posto dello 0 per lavorare sul video
i=0
# Load the input image.
image_height, image_width = [480,640]
crop_region = init_crop_region(image_height, image_width)
while cap.isOpened():
    ret,frame = cap.read()
    image = frame
    keypoints_with_scores = run_inference(
    movenet, image, crop_region,
    crop_size=[input_size, input_size])

    crop_region = determine_crop_region(
    keypoints_with_scores, image_height, image_width)
    i+=1
    print(i)
    print(keypoints_with_scores)
    draw_skeleton(frame, keypoints_with_scores, 0.3)
    cv2.imshow('MoveNet', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
print(keypoints_with_scores)

1
[[[[0.22254924 0.466552   0.01119886]
   [0.16021909 0.48569942 0.01049448]
   [0.16425435 0.4337352  0.01479992]
   [0.0578624  0.50930923 0.01132269]
   [0.00640325 0.42731696 0.03601051]
   [0.14922686 0.54559386 0.02283975]
   [0.13955402 0.3930812  0.01886869]
   [0.4029641  0.6034744  0.04927063]
   [0.4048142  0.35941833 0.02679753]
   [0.5978592  0.59896845 0.02368863]
   [0.5989799  0.4071526  0.01898695]
   [0.6671557  0.55481315 0.0386882 ]
   [0.66460913 0.42481658 0.03576315]
   [0.95527935 0.51815146 0.02569856]
   [0.95615125 0.45595294 0.02392762]
   [1.1631829  0.48427457 0.0073654 ]
   [1.1534737  0.44899553 0.00994345]]]]
2
[[[[0.7815005  0.15596706 0.03066462]
   [0.7558883  0.1679863  0.0199411 ]
   [0.75278527 0.14654389 0.01245651]
   [0.5831352  0.5672202  0.02585776]
   [0.4448576  0.6748111  0.01623358]
   [0.25909224 0.5604062  0.00582007]
   [0.2178632  0.59391135 0.02191005]
   [0.5809116  0.69183123 0.01811908]
   [0.41695943 0.6765084  0.04877412]
   [0

In [ ]:
def extract_all_keypoints(directory_path,actions):
     for file in actions:
        
        file_path = os.path.join(directory_path, file)

        video_list = []
        for f in os.listdir(file_path):
            name, ext = os.path.splitext(f)
            if ext == '.mp4':
                video_list.append(f)
    
        for video in video_list:
            
            video_path = os.path.join(file_path,video) 
            cap = cv2.VideoCapture(video_path)
            
            #Ottengo il numero di frames totali
            video_frames_count= int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
            #Calcolo l'intervallo dopo il quale i frames verranno aggiunti
            skip_frames_window= max(int(video_frames_count/45),1)#45 = sequence lenght
            
            video_height, video_width = [768, 1366]# sostituire con funzione
            crop_region = init_crop_region(video_height, video_width)
            
            i=1
        # iterando lungo i frames del video
            for frame_counter in range(45):
               
               #set la posizione corrente nel video
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
                
                #leggo il frame               
                ret,frame = cap.read()
                
                #se non viene letto esco
                if not ret:
                    break
                
                image = frame.copy()
                cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
                keypoints_with_scores = run_inference(
                movenet, image, crop_region,
                crop_size=[256, 256])# sostituire con variabile

                crop_region = determine_crop_region(
                keypoints_with_scores, video_height, video_width)
                
                keypoints = extract_keypoints(keypoints_with_scores)
               
                npy_path = os.path.join(file_path,video +"_landmarks",str(i))
                np.save(npy_path, keypoints)                                        
                i+=1
                print(i)
                print(npy_path)
                    

            cap.release()
            cv2.destroyAllWindows()
